In [1]:
import pandas as pd
import numpy as np 
import igraph as ig
import sys
from scipy.stats import zscore

In [2]:
path_list = []
with open("../../pathway/pathway_gene_sets/c2.cp.kegg.v2023.1.Hs.symbols.gmt") as gmt:
    for i in gmt:
        path_list.append([i.split()[0]] + i.split()[2:])

def read_g(sample):
    dftemp = pd.read_csv(f"sGCN354/sample_specific_{sample}.txt",sep="\t",header=None)
    dftemp.columns = [0,1,"weight"]
    dftemp["weight"] =abs(dftemp["weight"])
    all_genes = set(dftemp[0].tolist() + dftemp[1].tolist())
    sGCN = ig.Graph.TupleList(dftemp.itertuples(index=False),edge_attrs="weight")
    return sGCN, all_genes

def get_str(alist):
    return "_".join([str(i) for i in alist])


def get_sample_pair(sampleGroup):
    all_sample = []
    with open(f"{sampleGroup}") as f_w:
        for i in f_w:
            i= i.strip().split()
            all_sample.append(i[0])
    return all_sample

def trans_alist(alist):
    if type(alist) == int:
        return alist
    if type(alist) == float:
        return alist
    elif len(alist) > 0:
        thelist = pd.Series(alist)
        if thelist.sum() == 0:
            return 0
        else:
            #print(thelist.astype('float').sum(),thelist.count())
            return thelist.astype('float').sum()/thelist.count()
    else:
        return 0

In [3]:
import math
def cal_entropy(alist):
    h = 0
    if sum(alist) != 0:
        temp = sum(alist)
    for i in alist:
        if i == 0:
            h += 0
        else:
            h += (i/temp) * math.log2(i/temp)
    return -h

In [4]:
samplefile = "motzer.log2.354samples.filter.weight.txt"
sampleGroup = get_sample_pair(samplefile)

f_pathway = open(f"KEGGpathway_closeness_MOTZER354.txt","w")
for i in sampleGroup:
    g = read_g(i)
    allg_genes = g[1]
    expected = []
    expected.append(i)
    for j in path_list:
        left_genes = set(j[1:]).intersection(allg_genes)
    ##extract subgraph based pathway genes
        subg = g[0].subgraph(left_genes)
        total, genes_nx = len(j[1:]), len(left_genes)
        #print([t for t in subg.es])
        if genes_nx > 1:
            #pathentropy = cal_entropy([es["weight"] for es in subg.es])
            #eig = subg.eigenvector_centrality(directed=False, scale=False, weights="weight")
            eig =subg.closeness(mode='all', cutoff=None, weights="weight", normalized=True)
            #eig = subg.edge_betweenness(directed=False, cutoff=None, weights="weight")
            expected.append(str(trans_alist(eig)))
        else:
            expected.append(str(0))
    f_pathway.write("\t".join(expected)+"\n")
f_pathway.close()